In [ ]:
#| default_exp estimate

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import sys
import numpy as np
from fastcore.test import test_close, test_eq



In [ ]:
#| export

def legendre_polynomials(
    x: np.ndarray,      # points where the polynomials are to be evaluated
    max_deg: int,       # maximum degree
    a: float = -1.0,    # start of interval, classically -1
    b: float = 1.0,      # end of interval, classically 1
    no_constant: bool = False       # if True, delete the constant polynomial
    ) -> np.ndarray:    # returns an array of (max_deg+1) arrays of the shape of x
    """ evaluates the Legendre polynomials over x in the interval [a, b] """
    if a > np.min(x):
        sys.exit(f"legendre_polynomials: points below start of interval")
    if b <  np.max(x):
        sys.exit(f"legendre_polynomials: points above end of interval")
    p =np.zeros((*x.shape, max_deg + 1))
    p0 = np.ones_like(x)
    x_transf = 2.0*(x-a)/(b-a)-1.0
    p1 = x_transf
    p[:, :, 0], p[:, :, 1] = np.ones_like(x), x_transf
    for deg in range(2, max_deg+1):
        p2 = (2*deg-1)*(p[:, :, deg-1]*x_transf)-(deg-1)*p[:, :, deg-2]
        p[:, :, deg] = p2/deg
    polys_p = p[:, :, 1:] if no_constant else p
    return polys_p
        
    

def quantile_transform(
    v: np.ndarray        # a vector of counts
    ) -> np.ndarray:     # the corresponding quantiles
    """ transform a vector of counts into the corresponding quantiles """
    n = v.size
    q = np.zeros(n)
    for i in range(n):
        q[i] = np.sum(v <= v[i])/(n+1)
    return q
        


def generate_bases(
    nx: np.ndarray,           # the numbers of men of each type
    my: np.ndarray,           # the numbers of women of each type
    max_deg_x: int,              # the max degree for polynomials in `x`
    max_deg_y: int               # the max degree for polynomials in `y`
    ) -> np.ndarray:        # the matrix of base functions
    """ generates the bases for a semilinear specification """
    n_bases = 3 + max_deg_x * max_deg_y 
    n_types_men, n_types_women = nx.size, my.size
    types_men, types_women = np.arange(n_types_men), np.arange(n_types_women)
    base_funs = np.zeros((n_types_men, n_types_women, n_bases))
    base_funs[:, :, 0] = 1.0
    base_funs[:, :, 1] = np.outer(types_men, types_women, lambda x, y: x > y)
    base_funs[:, :, 2] = np.outer(types_men, types_women, lambda x, y: max(x-y, 0))
    # we quantile-transform nx and my
    q_nx = quantile_transform(nx)
    q_my = quantile_transform(my)
    # and we use the Legendre polynomials on [0,1], except 1
    polys_x = legendre_polynomials(q_nx, max_deg_x, a=0, no_constant=True)
    polys_y = legendre_polynomials(q_my, max_deg_y, a=0, no_constant=True)
    i_base = 3
    for deg_x in range(1, max_deg_x + 1):
        poly_x = polys_x[:, deg_x]
        for deg_y in range(1, max_deg_y +1):
            poly_y = polys_y[:, deg_y]
            base_funs[:, :, i_base] = np.outer(poly_x, poly_y)
            i_base += 1
    return base_funs
            
    
    
    
    
    

In [ ]:
#| hide
x= np.arange(6).reshape((2,3))/6
x_transf = 2*x-1
p3 = legendre_polynomials(x, 3, a=0)
test_eq(p3.shape, (2,3,4))

In [ ]:
#| hide
test_close(p3[:, :, 0], 1.0)

In [ ]:
#| hide
test_close(p3[:, :, 1], x_transf)

In [ ]:
#| hide
test_close(p3[:, :, 2], (3*x_transf*x_transf-1)/2)

In [ ]:
#| hide
test_close(p3[:, :, 3], (5*x_transf*x_transf*x_transf-3*x_transf)/2)

In [ ]:
#| hide
test_close(quantile_transform(np.array([2.0, -1.0, 3.5, 4.1, 1.2])), np.array([3, 1, 4, 5, 2])/6.0)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()